## Who is Tasty Bytes
![overview](https://quickstarts.snowflake.com/guide/zero_to_snowflake/img/a51f501137dea3c5.png)

In [ ]:
import streamlit as st
import altair as alt
from snowflake.snowpark.context import get_active_session
import pandas as pd

session = get_active_session()

# データの取得
df = session.table("TB_101.SYN_SCHEMA.ORDER_HEADER").to_pandas()

# 数値型と日時型のカラムを抽出
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
datetime_cols = df.select_dtypes(include=['datetime64']).columns

# 基本的な統計情報
st.header("ORDER_HEADERテーブルの基本統計")
col1, col2, col3, col4 = st.columns(4)
with col1:
    st.metric("総注文数", f"{len(df):,}")
with col2:
    st.metric("総売上", f"${df['ORDER_TOTAL'].sum():,.2f}")
with col3:
    st.metric("平均注文金額", f"${df['ORDER_TOTAL'].mean():,.2f}")
with col4:
    st.metric("ユニーク顧客数", f"{df['CUSTOMER_ID'].nunique():,}")

# 時系列分析
st.subheader("注文の時系列推移")
daily_orders = df.groupby(df['ORDER_TS'].dt.date).agg({
    'ORDER_ID': 'count',
    'ORDER_TOTAL': 'sum'
}).reset_index()

chart_type = st.selectbox(
    "表示する指標を選択",
    ["注文数", "売上高"]
)

if chart_type == "注文数":
    y_col = 'ORDER_ID'
    title = "日次注文数"
else:
    y_col = 'ORDER_TOTAL'
    title = "日次売上高"

line_chart = alt.Chart(daily_orders).mark_line().encode(
    x='ORDER_TS:T',
    y=alt.Y(y_col, title=title),
    tooltip=['ORDER_TS', y_col]
).properties(
    width=700,
    height=400,
    title=title
)
st.altair_chart(line_chart)